In [1]:
%matplotlib ipympl
from QDP import qdp
import os.path
import numpy as np
from scipy.stats import poisson
import matplotlib.pyplot as plt
from scipy import optimize
dp = '/home/ebert/jupyter-notebooks/FNode-data'

def expon(x, a, tau, b):
    return b + a*np.exp(-x/tau)

exclude = [2]

In [16]:
exp_name = ''
if exp_name:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
else:
    experiment_file = ''
    
single_only = False    

q = qdp.QDP(base_data_path=dp)
q.load_data_file(experiment_file)
print "ivars: {}".format(q.experiments[0]['variable_list'])
roi_map = ['cs', 'rb']
fmt_map = ['bo', 'rd']
cs_cuts = [[36],[3]]
rb_cuts = [[32],[3]]
marker_styles = [
    dict(color='cornflowerblue', marker='o', markeredgecolor='b', linestyle='None'),
    dict(color='indianred', marker='d', markeredgecolor='r', linestyle='None')
]
q.set_thresholds(cs_cuts, roi=0)
q.set_thresholds(rb_cuts, roi=1)
# q.generate_thresholds(loss=False)
print(q.get_thresholds())
retention = q.apply_thresholds(exclude_rois=exclude, ncondition=1)

ret = 'retention'
err = 'error'
if single_only:
    ret = 'conditional_retention'
    err = 'conditional_retention_err'

tbl_str = "ivar:\t{0}\tretention:{1} +- {2}"
entries = np.product(retention['retention'].shape[:-1])
for i in range(entries):
    print(tbl_str.format(
        retention['ivar'].reshape((entries,-1))[i][0],
        retention[ret].reshape((entries,-1))[i],
        retention[err].reshape((entries,-1))[i]
    ))
q.save_experiment_data()
q.experiments[0]['variable_list']
xlab = q.experiments[0]['variable_desc'][q.experiments[0]['variable_list'][0]]['description']

# plotting
fig, ax = plt.subplots(nrows=2, figsize=(5,8))
plt.subplots_adjust(wspace=0.3)
for i, yvals in enumerate([ret, 'loading']):
    for e in range(len(retention['ivar'])):
        for r in range(2):            
            if yvals==ret:
                try:
                    ax[i].errorbar(retention['ivar'][e, :, r], retention[yvals][e, :, r],
                        yerr=retention[err][e,:,r],
                        **marker_styles[r]
                    )
                except Exception as ee:
                    print(ee)
                    ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], **marker_styles[r])
                # fitting
                xs = np.linspace(min(retention['ivar'][e, :, 0]), max(retention['ivar'][e, :, 0]),100)
                guess = [0.5, 10, 0.5]
                try:
                    popt, pcov = optimize.curve_fit(
                        expon,
                        retention['ivar'][e, :, 0],
                        retention[yvals][e, :, r],
                        p0=guess,
                        sigma=retention[err][e, :, r]
                    )
                except Exception as ee:
                    print ee
                else:
                    ax[i].plot(xs, expon(xs, *popt), fmt_map[r][0]+'--', alpha=0.6)
                    print(popt)
                    fit_data = "{}\na: {:.3f}\ntau: {:.3f}\nb: {:.3f}"
                    ax[i].text(0.2+r*max(xs)*0.7, 0.1, fit_data.format(roi_map[r], *popt), fontsize=12)
            else:
                ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], **marker_styles[r])
                
        
    ax[i].set_ylim(0., 1)
    ax[i].set_xlabel(xlab)
    ax[i].set_ylabel(yvals)

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'exponential_retention_fit.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

data at: 2018_07_20/2018_07_20_19_09_16_csrb-31-t1/results.hdf5
name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
ivars: [u'gap_time']
{0: [[36], [3]], 1: [[32], [3]]}
ivar:	8.5	retention:[ 0.          0.          2.49364121] +- [ 0.00759959  0.00759959  0.00759959]
ivar:	1006.8	retention:[ 0.12598425  0.025       5.        ] +- [ 0.01467991  0.01467991  0.01467991]
ivar:	2005.1	retention:[  1.34146341e-01   0.00000000e+00   5.00000000e+03] +- [ 0.00741019  0.00741019  0.00741019]
ivar:	3003.4	retention:[ 0.23333333  0.          5.        ] +- [ 0.00779891  0.00779891  0.00779891]
ivar:	4001.7	retention:[  2.23880597e-01   1.35135135e-02   5.00000000e+03] +- [ 0.01128781  0.01128781  0.01128781]
ivar:	5000.0	retention:[ 0.28688525  0.01298701  5.        ] +- [ 0.01085092  0.01085092  0.01085092]


FigureCanvasNbAgg()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in exp
  # This is added back by InteractiveShellApp.init_path()


[ -0.39189966  12.51946449   0.1987534 ]
[ 0.22209468  0.23466575  0.00455617]


In [17]:
# plotting
offset = 0.5
single_only = True

ret = 'retention'
err = 'error'
if single_only:
    ret = 'conditional_retention'
    err = 'conditional_retention_err'

fig, ax = plt.subplots(nrows=2, figsize=(5,8))
plt.subplots_adjust(wspace=0.3)
for i, yvals in enumerate([ret, 'loading']):
    for e in range(len(retention['ivar'])):
        for r in range(2):            
            if yvals==ret:
                try:
                    ax[i].errorbar(retention['ivar'][e, :, r], retention[yvals][e, :, r],
                        yerr=retention[err][e,:,r],
                        fmt=fmt_map[r]
                    )
                except Exception as ee:
                    print(ee)
                    ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], fmt_map[r])
                # fitting
                xs = np.linspace(min(retention['ivar'][e, :, 0]), max(retention['ivar'][e, :, 0]),100)
                guess = [-0.5, 1e3, 0]
                try:
                    popt, pcov = optimize.curve_fit(
                        lambda x, a, tau: expon(x, a, tau, offset),
                        retention['ivar'][e, :, 0],
                        retention[yvals][e, :, r],
                        p0=guess[:2],
                        sigma=retention[err][e, :, r]
                    )
                except Exception as ee:
                    print ee
                else:
                    ax[i].plot(xs, expon(xs,popt[0],popt[1],offset), fmt_map[r][0]+'--', alpha=0.6)
                    print(popt)
                    print(np.sqrt(np.diagonal(np.abs(pcov))))
                    fit_data = "{}\na: {:.3f}\ntau: {:.3f}\nb: {:.3f}"
                    ax[i].text(0.2+r*0.6*max(xs), 0.1, fit_data.format(roi_map[r],popt[0],popt[1],offset), fontsize=12)
            else:
                ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], fmt_map[r])
                
        
    ax[i].set_ylim(0.,0.6)
    ax[i].set_xlabel(xlab)
    ax[i].set_ylabel(yvals)

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'exponential_retention_fit.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

[ -4.93128567e-01   5.15284255e+03]
[  4.44422866e-02   1.11242545e+03]
[ -4.99379288e-01   2.32230555e+05]
[  8.55663559e-03   3.17455689e+05]


In [36]:
(1./3.21-1./6.2)**-1

6.656187290969901

In [4]:
retention[err][0,:,0]

array([ 0.00940178,  0.01283855,  0.01283855,  0.01264552,  0.01389937,
        0.01345468])